In [1]:
import os
import mlflow


# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

mlflow.set_tracking_uri(f'http://0.0.0.0:5000')

In [2]:
import psycopg
import pandas as pd

In [3]:
DB_DESTINATION_USER = os.getenv("DB_DESTINATION_USER")
DB_DESTINATION_PASSWORD = os.getenv("DB_DESTINATION_PASSWORD")
DB_DESTINATION_HOST = os.getenv("DB_DESTINATION_HOST")
DB_DESTINATION_PORT = os.getenv("DB_DESTINATION_PORT")
DB_DESTINATION_NAME = os.getenv("DB_DESTINATION_NAME")

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": DB_DESTINATION_HOST, 
    "port": DB_DESTINATION_PORT,
    "dbname": DB_DESTINATION_NAME,
    "user": DB_DESTINATION_USER,
    "password": DB_DESTINATION_PASSWORD,
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns) 

In [4]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,17,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,...,None,None,None,None,Female,0,No,No,No,0
1,21,8779-QRDMV,2019-11-01,2019-12-01,Month-to-month,Yes,Electronic check,39.65,39.65,DSL,...,Yes,No,No,Yes,Male,1,No,No,None,1
2,22,1680-VDCWW,2019-02-01,NaT,One year,No,Bank transfer (automatic),19.80,202.25,None,...,None,None,None,None,Male,0,Yes,No,No,0
3,23,1066-JKSGK,2019-11-01,2019-12-01,Month-to-month,No,Mailed check,20.15,20.15,None,...,None,None,None,None,Male,0,No,No,No,1
4,24,3638-WEABW,2015-04-01,NaT,Two year,Yes,Credit card (automatic),59.90,3505.10,DSL,...,No,Yes,No,No,Female,0,Yes,No,Yes,0


In [5]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
    # посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

    # обновите словарь stats
    stats.update(column_stat)


stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max() # посчитайте максимальное значение в колонке
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isna().sum() # посчитайте кол-во пустых строк в колонке

In [6]:
stats

{'type_Month-to-month': 3875,
 'type_Two year': 1695,
 'type_One year': 1473,
 'paperless_billing_Yes': 4171,
 'paperless_billing_No': 2872,
 'internet_service_Fiber optic': 3096,
 'internet_service_DSL': 2421,
 'online_security_No': 3498,
 'online_security_Yes': 2019,
 'online_backup_No': 3088,
 'online_backup_Yes': 2429,
 'device_protection_No': 3095,
 'device_protection_Yes': 2422,
 'tech_support_No': 3473,
 'tech_support_Yes': 2044,
 'streaming_tv_No': 2810,
 'streaming_tv_Yes': 2707,
 'streaming_movies_No': 2785,
 'streaming_movies_Yes': 2732,
 'gender_Male': 3555,
 'gender_Female': 3488,
 'senior_citizen_0': 5901,
 'senior_citizen_1': 1142,
 'partner_No': 3641,
 'partner_Yes': 3402,
 'dependents_No': 4933,
 'dependents_Yes': 2110,
 'multiple_lines_No': 3390,
 'multiple_lines_Yes': 2971,
 'target_0': 5174,
 'target_1': 1869,
 'data_length': 7043,
 'monthly_charges_min': 18.25,
 'monthly_charges_max': 118.75,
 'monthly_charges_mean': 64.76169246059918,
 'monthly_charges_median': 70

In [7]:
from sklearn.model_selection import train_test_split

df_cleared = df.copy()

df_categorical_col = df.select_dtypes(include=['object', 'category']).columns
df_categorical = df_cleared[df_categorical_col].fillna('')

df_cleared[df_categorical_col] = df_categorical

y = df_cleared['target']
X = df_cleared[df_cleared.select_dtypes(exclude=['datetime64']).columns].drop(['target'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [21]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=300)

model.fit(X_train, y_train, cat_features=list(X.select_dtypes(include=['object', 'category'])))

Learning rate set to 0.060271
0:	learn: 0.6556416	total: 54.5ms	remaining: 16.3s
1:	learn: 0.6274885	total: 114ms	remaining: 16.9s
2:	learn: 0.5994344	total: 187ms	remaining: 18.5s
3:	learn: 0.5777254	total: 253ms	remaining: 18.7s
4:	learn: 0.5611215	total: 284ms	remaining: 16.8s
5:	learn: 0.5438758	total: 370ms	remaining: 18.1s
6:	learn: 0.5287493	total: 435ms	remaining: 18.2s
7:	learn: 0.5144786	total: 509ms	remaining: 18.6s
8:	learn: 0.5053947	total: 593ms	remaining: 19.2s
9:	learn: 0.4939980	total: 686ms	remaining: 19.9s
10:	learn: 0.4844287	total: 763ms	remaining: 20s
11:	learn: 0.4775988	total: 822ms	remaining: 19.7s
12:	learn: 0.4719583	total: 868ms	remaining: 19.2s
13:	learn: 0.4653080	total: 938ms	remaining: 19.2s
14:	learn: 0.4593295	total: 984ms	remaining: 18.7s
15:	learn: 0.4551172	total: 1.04s	remaining: 18.5s
16:	learn: 0.4513687	total: 1.1s	remaining: 18.4s
17:	learn: 0.4472413	total: 1.15s	remaining: 18s
18:	learn: 0.4437202	total: 1.21s	remaining: 17.9s
19:	learn: 0.44

In [22]:
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, log_loss

prediction = model.predict(X_test)
probas = model.predict_proba(X_test)[:, -1]
# импортируйте необходимые вам модули

# заведите словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода
_, err1, _, err2 = _, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel() # ваш код здесь #
auc = roc_auc_score(y_test, probas) # ваш код здесь #
precision = precision_score(y_test, prediction) # ваш код здесь #
recall = recall_score(y_test, prediction) # ваш код здесь #
f1 = f1_score(y_test, prediction) # ваш код здесь #
logloss = log_loss(y_test, prediction) # ваш код здесь #

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

In [23]:
metrics

{'err1': 0.06881720430107527,
 'err2': 0.13720430107526882,
 'auc': 0.8519906883732062,
 'precision': 0.6659707724425887,
 'recall': 0.5307820299500832,
 'f1': 0.5907407407407407,
 'logloss': 6.852169805587002}

In [24]:
EXPERIMENT_NAME = "churn_experiment"
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_nikolaistepanov"

pip_requirements = './requirements.txt'
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}

if mlflow.get_experiment_by_name(EXPERIMENT_NAME):
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
else:
    experiment_id = mlflow.create_experiment(name=EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.log_params({"iterations": 300})
    mlflow.log_metrics(metrics)

    model_info = mlflow.catboost.log_model( 
        cb_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        metadata=metadata,
        await_registration_for=60,
    )

/home/mle-user/mle_projects/mle-mlflow/.venv-mle-mlflow/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'churn_model_nikolaistepanov' already exists. Creating a new version of this model...
2024/05/04 21:56:52 INFO mlflow.track